In [48]:
from matplotlib import colors, cm
import matplotlib.pyplot as plt
import scipy.io
import itertools
from skimage import io, filters, morphology, restoration, feature, transform
from matplotlib.collections import LineCollection
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from itertools import cycle
import sklearn.cluster
from sklearn.preprocessing import MinMaxScaler
import skimage.segmentation as seg
import scipy as sc

import numpy as np
import math
import numpy.ma as ma

#import nglpy_cuda as ngl
import nglpy as ngl
import topopy
#import pdir
import time
import os
import utpy.utils
import utpy.vis
import flatpy
from functools import partial
from itertools import cycle
from evtk.hl import imageToVTK
import random
import time
from time import process_time
from numpy import linalg as LA

import vtk
from vtk.util import numpy_support
import os
import sys

In [49]:
def readDS(fname) :
    reader = vtk.vtkDataSetReader()
    reader.SetFileName(fname)
    reader.ReadAllVectorsOn()
    reader.ReadAllScalarsOn()
    reader.Update()
    ds = reader.GetOutput()
    return ds

In [51]:
filename = "beetle_496_832_832.vtk"
fieldarray= "ground_truth"
ds = readDS(filename)
#print(ds)

#get vtkarray
point_array = ds.GetPointData().GetArray(fieldarray)
#print(point_array)

#get numpy array
nparray = numpy_support.vtk_to_numpy(point_array)
ground_truth = nparray.reshape((496,832,832))

print(ground_truth.shape)

(496, 832, 832)


In [2]:
## Load ground truth
#myarray = np.fromfile('stagbeetle832x832x494.dat', dtype='H')

#print(len(myarray))

#newarray = myarray[3: len(myarray)]

#ground_truth = newarray.reshape((494, 832, 832))

##newarray.astype('H').tofile('beetle832x832x484.raw')

##print(832*832*494)


341958659


In [52]:
# Preprare ensemble from the ground truth

ensemble = np.zeros((124,208,208,64),dtype='H')

for i in range(0,496,4):
    for j in range(0,832,4):
        for k in range(0,832,4):
            
            l = int(i/4)
            m = int(j/4)
            n = int(k/4)
            
            count = 0
            for n1 in range(4):
                for n2 in range(4):
                    for n3 in range(4):
                        ensemble[l,m,n,count] = ground_truth[i+n1,j+n2,k+n3]
                        count += 1
            


In [4]:
# Compute the probabillity of values being less than the isovalue and greater than the isovalue at each grid vertex
def computePositiveNegativeProbabilities(ensemble, isovalue, confidence, distributionModel):
    h, w, d, numMembers = ensemble.shape
    minimumGrid = np.zeros((h,w,d))
    maximumGrid = np.zeros((h,w,d))
    meanGrid = np.zeros((h,w,d))
    mostProbableGrid = np.zeros((h,w,d))
    
    meanGaussian = np.zeros((h,w,d))
    stdGaussian = np.zeros((h,w,d))
    for i in range(h):
        for j in range(w):
            for k in range(d):
                minimum = math.inf
                maximum = -math.inf
                t = np.array([])
                for idx in range(numMembers):
                    t = np.append(t, [ensemble[i,j,k,idx]])
                    if(ensemble[i,j,k,idx] < minimum):
                        minimum = ensemble[i,j,k,idx]
                    if(ensemble[i,j,k,idx] > maximum):
                        maximum = ensemble[i,j,k,idx]    
                tMean = np.mean(t,0)
                tVar = np.var(t,0)
                tStd = math.sqrt(tVar)
            
                #Uniform
                minimumGrid[i,j,k] = minimum
                maximumGrid[i,j,k] = maximum   
            
                #Gaussian
                meanGaussian[i,j,k] = tMean
                stdGaussian[i,j,k] = tStd
            
    # Compute probabilitis based of confidence value (between 0 and 100)
    for i in range(h):
        for j in range(w):
            for k in range(d):
            
                minimum=minimumGrid[i,j,k]
                maximum=maximumGrid[i,j,k]
            
                minimumGrid[i,j,k]=minimum
                maximumGrid[i,j,k]=maximum
            
                meanVal = (maximum + minimum)/2
            
                confidenceFraction = 0.01*confidence*(meanVal - minimum)
            
                newMinimum = meanVal - confidenceFraction
                newMaximum = meanVal + confidenceFraction
            
                # Revise minimum and maximum grids based on confidence information
                minimumGrid[i,j,k]=newMinimum
                maximumGrid[i,j,k]=newMaximum
                meanGrid[i,j,k] = (newMinimum + newMaximum)/2
    
                #plt.figure()
                #plt.imshow(minimumGrid)
                #plt.contour(minimumGrid, levels=[isovalue])
    
                #plt.figure()
                #plt.imshow(maximumGrid)
                #plt.contour(maximumGrid, levels=[isovalue])
    
    # Compute negative and positive probabilities
    # Pr(value <= isovalue)
    negativeProbabilitiesGrid = np.zeros((h,w,d))
    # Pr(value > isovalue)
    positiveProbabilitiesGrid = np.zeros((h,w,d))
    for i in range(h):
        for j in range(w):
            for k in range(d):
                minimum = minimumGrid[i,j,k]
                maximum = maximumGrid[i,j,k]
                meanVal = meanGrid[i,j,k]
                if (isovalue <= minimum):
                    positiveProbabilitiesGrid[i,j,k] = 1.0
                    negativeProbabilitiesGrid[i,j,k] = 0.0
                elif (isovalue >= maximum):    
                    positiveProbabilitiesGrid[i,j,k] = 0.0
                    negativeProbabilitiesGrid[i,j,k] = 1.0
                else:
                    if distributionModel == 'uniform':
                        positiveProbabilitiesGrid[i,j,k] = (maximum - isovalue)/(maximum - minimum)
                        negativeProbabilitiesGrid[i,j,k] = 1.0 - positiveProbabilitiesGrid[i,j,k]
                    elif distributionModel == 'Gaussian':
                        tempMean = meanGaussian[i,j,k]
                        tempStd = stdGaussian[i,j,k]
                        negativeProbabilitiesGrid[i,j,k] = 0.5*(1 + math.erf((isovalue - tempMean)/(math.sqrt(2)*tempStd)))
                        positiveProbabilitiesGrid[i,j,k] = 1.0 - negativeProbabilitiesGrid[i,j,k]
                    else:
                        print('No valid model')
            
                # Compute most probable grid
                if(positiveProbabilitiesGrid[i,j,k] > negativeProbabilitiesGrid[i,j,k]):
                    mostProbableGrid[i,j,k] = (meanVal + maximum)/2
                else:
                    mostProbableGrid[i,j,k] = (minimum + meanVal)/2  
            
    return positiveProbabilitiesGrid, negativeProbabilitiesGrid, maximumGrid, minimumGrid, mostProbableGrid

In [5]:
def computeMSTopologyCasesProbabilities(positiveProbabilitiesGrid, negativeProbabilitiesGrid):
    h, w, d = positiveProbabilitiesGrid.shape
    MStopologyCasesProbabilities = np.zeros((h-1,w-1,d-1, 256))
    # Record number of cases per cell as well
    numCases = np.zeros((h-1,w-1,d-1))

    tempNumCases = 1
    for i in range(h-1):
        for j in range(w-1):
            for k in range(d-1):
            
                x1Positive = positiveProbabilitiesGrid[i,j,k]
                x1Negative = negativeProbabilitiesGrid[i,j,k]
                x1Sign = np.array([x1Negative, x1Positive])
                if(x1Positive > 0) and (x1Positive < 1):
                    tempNumCases = tempNumCases*2
            
                x2Positive = positiveProbabilitiesGrid[i+1,j,k]
                x2Negative = negativeProbabilitiesGrid[i+1,j,k]
                x2Sign = np.array([x2Negative, x2Positive])
                if(x2Positive > 0) and (x2Positive < 1):
                    tempNumCases = tempNumCases*2
            
                x3Positive = positiveProbabilitiesGrid[i+1,j+1,k]
                x3Negative = negativeProbabilitiesGrid[i+1,j+1,k]
                x3Sign = np.array([x3Negative, x3Positive])
                if(x3Positive > 0) and (x3Positive < 1):
                    tempNumCases = tempNumCases*2
            
                x4Positive = positiveProbabilitiesGrid[i,j+1,k]
                x4Negative = negativeProbabilitiesGrid[i,j+1,k]
                x4Sign = np.array([x4Negative, x4Positive])
                if(x4Positive > 0) and (x4Positive < 1):
                    tempNumCases = tempNumCases*2
                    
                x5Positive = positiveProbabilitiesGrid[i,j,k+1]
                x5Negative = negativeProbabilitiesGrid[i,j,k+1]
                x5Sign = np.array([x5Negative, x5Positive])
                if(x5Positive > 0) and (x5Positive < 1):
                    tempNumCases = tempNumCases*2
            
                x6Positive = positiveProbabilitiesGrid[i+1,j,k+1]
                x6Negative = negativeProbabilitiesGrid[i+1,j,k+1]
                x6Sign = np.array([x6Negative, x6Positive])
                if(x6Positive > 0) and (x6Positive < 1):
                    tempNumCases = tempNumCases*2
            
                x7Positive = positiveProbabilitiesGrid[i+1,j+1,k+1]
                x7Negative = negativeProbabilitiesGrid[i+1,j+1,k+1]
                x7Sign = np.array([x7Negative, x7Positive])
                if(x7Positive > 0) and (x7Positive < 1):
                    tempNumCases = tempNumCases*2
            
                x8Positive = positiveProbabilitiesGrid[i,j+1,k+1]
                x8Negative = negativeProbabilitiesGrid[i,j+1,k+1]
                x8Sign = np.array([x8Negative, x8Positive])
                if(x8Positive > 0) and (x8Positive < 1):
                    tempNumCases = tempNumCases*2
                
                numCases[i,j,k] = tempNumCases
                # Reset tempNumCases for next iteration
                tempNumCases = 1
                
                count = 0
                for s1 in range(2):
                    for s2 in range(2):
                        for s3 in range(2):
                            for s4 in range(2):
                                for s5 in range(2):
                                    for s6 in range(2):
                                        for s7 in range(2):
                                            for s8 in range(2):
                                                MStopologyCasesProbabilities[i,j,k,count] = x1Sign[s1]*x2Sign[s2]*x3Sign[s3]*x4Sign[s4]*x5Sign[s5]*x6Sign[s6]*x7Sign[s7]*x8Sign[s8]
                                                count+=1
              
            
    # Compute entropy of probability distribution
    entropy = np.zeros((h-1,w-1,d-1))
        
    #for i in range(256):
        
    #    probs = MStopologyCasesProbabilities[:,:,:,i]
    #    res = ma.log2(probs);
    #    entropy += np.multiply(-probs,res)
        
        
    for i in range(h-1):
        for j in range(w-1):
            for k in range(d-1):
                entropyValue = 0
                res = 0
                for num in range(256):
                    probs = MStopologyCasesProbabilities[i,j,k,num]
                    if probs > 0:
                        res = ma.log2(probs)
                    entropyValue += np.multiply(-probs,res)    
                entropy[i,j,k] = entropyValue
        
    return MStopologyCasesProbabilities, entropy, numCases

In [6]:
def probabilisticMarchingSquares(positiveProbabilitiesGrid, negativeProbabilitiesGrid):
    
    h, w, d = positiveProbabilitiesGrid.shape
    # Record number of cases per cell as well
    crossingProbabilities = np.zeros((h-1,w-1,d-1))
    for i in range(h-1):
        for j in range(w-1):
            for k in range(d-1):
                nonCrossingProbability1 = positiveProbabilitiesGrid[i,j,k]*positiveProbabilitiesGrid[i+1,j,k]*positiveProbabilitiesGrid[i,j+1,k]*positiveProbabilitiesGrid[i+1,j+1,k]*positiveProbabilitiesGrid[i,j,k+1]*positiveProbabilitiesGrid[i+1,j,k+1]*positiveProbabilitiesGrid[i,j+1,k+1]*positiveProbabilitiesGrid[i+1,j+1,k+1]
                nonCrossingProbability2 = negativeProbabilitiesGrid[i,j,k]*negativeProbabilitiesGrid[i+1,j,k]*negativeProbabilitiesGrid[i,j+1,k]*negativeProbabilitiesGrid[i+1,j+1,k]*negativeProbabilitiesGrid[i,j,k+1]*negativeProbabilitiesGrid[i+1,j,k+1]*negativeProbabilitiesGrid[i,j+1,k+1]*negativeProbabilitiesGrid[i+1,j+1,k+1]
                nonCrossingProbability = nonCrossingProbability1 + nonCrossingProbability2
                crossingProbability = 1.0 - nonCrossingProbability
                crossingProbabilities[i,j,k] = crossingProbability
    return crossingProbabilities

In [7]:
def getIndependentModelProb(meanArray, covMatrix, isovalue):
    
        m1 = meanArray[0]
        m2 = meanArray[1]
        m3 = meanArray[2]
        m4 = meanArray[3]
        m5 = meanArray[4]
        m6 = meanArray[5]
        m7 = meanArray[6]
        m8 = meanArray[7]
        
        std1 = np.sqrt(covMatrix[0,0])
        std2 = np.sqrt(covMatrix[1,1])
        std3 = np.sqrt(covMatrix[2,2])
        std4 = np.sqrt(covMatrix[3,3])
        std5 = np.sqrt(covMatrix[4,4])
        std6 = np.sqrt(covMatrix[5,5])
        std7 = np.sqrt(covMatrix[6,6])
        std8 = np.sqrt(covMatrix[7,7])
        
        negProb1 = 0.5*(1 + math.erf((isovalue - m1)/(math.sqrt(2)*std1)))
        negProb2 = 0.5*(1 + math.erf((isovalue - m2)/(math.sqrt(2)*std2)))
        negProb3 = 0.5*(1 + math.erf((isovalue - m3)/(math.sqrt(2)*std3)))
        negProb4 = 0.5*(1 + math.erf((isovalue - m4)/(math.sqrt(2)*std4)))
        negProb5 = 0.5*(1 + math.erf((isovalue - m5)/(math.sqrt(2)*std5)))
        negProb6 = 0.5*(1 + math.erf((isovalue - m6)/(math.sqrt(2)*std6)))
        negProb7 = 0.5*(1 + math.erf((isovalue - m7)/(math.sqrt(2)*std7)))
        negProb8 = 0.5*(1 + math.erf((isovalue - m8)/(math.sqrt(2)*std8)))
        
        posProb1 = 1 - negProb1
        posProb2 = 1 - negProb2
        posProb3 = 1 - negProb3
        posProb4 = 1 - negProb4
        posProb5 = 1 - negProb5
        posProb6 = 1 - negProb6
        posProb7 = 1 - negProb7
        posProb8 = 1 - negProb8
        
        allNegativeProb = negProb1*negProb2*negProb3*negProb4*negProb5*negProb6*negProb7*negProb8
        allPositiveProb = posProb1*posProb2*posProb3*posProb4*posProb5*posProb6*posProb7*posProb8
        
        crossingProb = 1 - allNegativeProb -allPositiveProb
        
        return crossingProb

In [8]:
def PearsonAdaptiveCrossingProbWithEigenDecomposition(meanArray, covMatrix, isovalue, numSamples):
    
    w, v = LA.eig(covMatrix)
    
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]
    
    #print('eigenvalues:',w)
        
    indexImpEigenvalue = 0
    threshold2 = 0.01
        
    # search important eigenvalue from end. Not checked for w[3] as it would have use independent model if
    # w[3] > 0.2*w[0]
        
    if (w[7] > threshold2*w[0]):
        indexImpEigenvalue = 7
        
    elif (w[6] > threshold2*w[0]):
        indexImpEigenvalue = 6
            
    elif (w[5] > threshold2*w[0]):
        indexImpEigenvalue = 5
            
    elif (w[4] > threshold2*w[0]):
        indexImpEigenvalue = 4
            
    elif (w[3] > threshold2*w[0]):
        indexImpEigenvalue = 3
        
    elif (w[2] > threshold2*w[0]):
        indexImpEigenvalue = 2
        
    elif (w[1] > threshold2*w[0]):
        indexImpEigenvalue = 1
            
        
    # Transformed isovalue
    transformedIso1 = (isovalue-meanArray[0]) 
    transformedIso2 = (isovalue-meanArray[1])    
    transformedIso3 = (isovalue-meanArray[2]) 
    transformedIso4 = (isovalue-meanArray[3])
    transformedIso5 = (isovalue-meanArray[4]) 
    transformedIso6 = (isovalue-meanArray[5])    
    transformedIso7 = (isovalue-meanArray[6]) 
    transformedIso8 = (isovalue-meanArray[7]) 
    
    xTest = np.zeros((numSamples))
    yTest = np.zeros((numSamples))
    zTest = np.zeros((numSamples))
    wTest = np.zeros((numSamples))
    aTest = np.zeros((numSamples))
    bTest = np.zeros((numSamples))
    cTest = np.zeros((numSamples))
    dTest = np.zeros((numSamples))
    
    for i in range(indexImpEigenvalue+1):

        temp = np.random.normal(0, math.sqrt(w[i]), numSamples).T
        
        
        xTest = xTest + v[0][i]*temp
        yTest = yTest + v[1][i]*temp
        zTest = zTest + v[2][i]*temp
        wTest = wTest + v[3][i]*temp
        aTest = aTest + v[4][i]*temp
        bTest = bTest + v[5][i]*temp
        cTest = cTest + v[6][i]*temp
        dTest = dTest + v[7][i]*temp

            
    numNegativeNonCrossing = 0
    for i in range(numSamples):
        if ((xTest[i]<=transformedIso1) and (yTest[i]<=transformedIso2) and (zTest[i]<=transformedIso3) and (wTest[i]<=transformedIso4) and (aTest[i]<=transformedIso5) and (bTest[i]<=transformedIso6) and (cTest[i]<=transformedIso7) and (dTest[i]<=transformedIso8)):
            numNegativeNonCrossing = numNegativeNonCrossing + 1
        
    numPositiveNonCrossing = 0
    for i in range(numSamples):
        if ((xTest[i]>=transformedIso1) and (yTest[i]>=transformedIso2) and (zTest[i]>=transformedIso3) and (wTest[i]>=transformedIso4) and (aTest[i]>=transformedIso5) and (bTest[i]>=transformedIso6) and (cTest[i]>=transformedIso7) and (dTest[i]>=transformedIso8)):
            numPositiveNonCrossing = numPositiveNonCrossing + 1
    
    crossingProb = 1.0 - (numNegativeNonCrossing/numSamples) - (numPositiveNonCrossing/numSamples)
    
    return crossingProb

In [39]:
def adaptiveCrossingProbWithEigenDecomposition(meanArray, covMatrix, isovalue, numSamples):
    
    w, v = LA.eig(covMatrix)
    
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]
    
    #print('eigenvalues:',w)
    indexImpEigenvalue = 0
    threshold2 = 0
        
    # search important eigenvalue from end. Not checked for w[3] as it would have use independent model if
    # w[3] > 0.2*w[0]
        
    if (w[7] > threshold2*w[0]):
        indexImpEigenvalue = 7
        
    elif (w[6] > threshold2*w[0]):
        indexImpEigenvalue = 6
            
    elif (w[5] > threshold2*w[0]):
        indexImpEigenvalue = 5
            
    elif (w[4] > threshold2*w[0]):
        indexImpEigenvalue = 4
            
    elif (w[3] > threshold2*w[0]):
        indexImpEigenvalue = 3
        
    elif (w[2] > threshold2*w[0]):
        indexImpEigenvalue = 2
        
    elif (w[1] > threshold2*w[0]):
        indexImpEigenvalue = 1
            
        
    # Transformed isovalue
    transformedIso1 = (isovalue-meanArray[0]) 
    transformedIso2 = (isovalue-meanArray[1])    
    transformedIso3 = (isovalue-meanArray[2]) 
    transformedIso4 = (isovalue-meanArray[3])
    transformedIso5 = (isovalue-meanArray[4]) 
    transformedIso6 = (isovalue-meanArray[5])    
    transformedIso7 = (isovalue-meanArray[6]) 
    transformedIso8 = (isovalue-meanArray[7]) 
    
    xTest = np.zeros((numSamples))
    yTest = np.zeros((numSamples))
    zTest = np.zeros((numSamples))
    wTest = np.zeros((numSamples))
    aTest = np.zeros((numSamples))
    bTest = np.zeros((numSamples))
    cTest = np.zeros((numSamples))
    dTest = np.zeros((numSamples))
    
    for i in range(indexImpEigenvalue+1):

        temp = np.random.normal(0, math.sqrt(w[i]), numSamples).T
        
        
        xTest = xTest + v[0][i]*temp
        yTest = yTest + v[1][i]*temp
        zTest = zTest + v[2][i]*temp
        wTest = wTest + v[3][i]*temp
        aTest = aTest + v[4][i]*temp
        bTest = bTest + v[5][i]*temp
        cTest = cTest + v[6][i]*temp
        dTest = dTest + v[7][i]*temp

            
    numNegativeNonCrossing = 0
    for i in range(numSamples):
        if ((xTest[i]<=transformedIso1) and (yTest[i]<=transformedIso2) and (zTest[i]<=transformedIso3) and (wTest[i]<=transformedIso4) and (aTest[i]<=transformedIso5) and (bTest[i]<=transformedIso6) and (cTest[i]<=transformedIso7) and (dTest[i]<=transformedIso8)):
            numNegativeNonCrossing = numNegativeNonCrossing + 1
        
    numPositiveNonCrossing = 0
    for i in range(numSamples):
        if ((xTest[i]>=transformedIso1) and (yTest[i]>=transformedIso2) and (zTest[i]>=transformedIso3) and (wTest[i]>=transformedIso4) and (aTest[i]>=transformedIso5) and (bTest[i]>=transformedIso6) and (cTest[i]>=transformedIso7) and (dTest[i]>=transformedIso8)):
            numPositiveNonCrossing = numPositiveNonCrossing + 1
    
    crossingProb = 1.0 - (numNegativeNonCrossing/numSamples) - (numPositiveNonCrossing/numSamples)
    
    return crossingProb

In [10]:
def getCrossingProbWithEigenDecomposition(meanArray, covMatrix, isovalue, numSamples):
    
    w, v = LA.eig(covMatrix)
    
    idx = w.argsort()[::-1]   
    w = w[idx]
    v = v[:,idx]
    
    #print('eigenvalues:',w)

    # Transformed isovalue
    transformedIso1 = (isovalue-meanArray[0]) 
    transformedIso2 = (isovalue-meanArray[1])    
    transformedIso3 = (isovalue-meanArray[2]) 
    transformedIso4 = (isovalue-meanArray[3])  
    transformedIso5 = (isovalue-meanArray[4]) 
    transformedIso6 = (isovalue-meanArray[5])    
    transformedIso7 = (isovalue-meanArray[6]) 
    transformedIso8 = (isovalue-meanArray[7])  
    
    #numSamples = 500
    
    # Find index until which eigenvalue is greater than 200
    k=1
    #for k in range(8):
    #    if(w[k]<10000):
    #        break
   
    #if (k > 0):
    #    numSamples = k*64
    #else:
    #    numSamples = 64

    xTest = np.zeros((numSamples))
    yTest = np.zeros((numSamples))
    zTest = np.zeros((numSamples))
    wTest = np.zeros((numSamples))
    aTest = np.zeros((numSamples))
    bTest = np.zeros((numSamples))
    cTest = np.zeros((numSamples))
    dTest = np.zeros((numSamples))
    
    #print(xTest.shape)
    
    #xD = np.random.normal(0, math.sqrt(w[0]), numSamples).T
    
    #print(xD.shape)
    
    for i in range(k):

        temp = np.random.normal(0, math.sqrt(w[i]), numSamples).T
        
        
        xTest = xTest + v[0][i]*temp
        yTest = yTest + v[1][i]*temp
        zTest = zTest + v[2][i]*temp
        wTest = wTest + v[3][i]*temp
        aTest = aTest + v[4][i]*temp
        bTest = bTest + v[5][i]*temp
        cTest = cTest + v[6][i]*temp
        dTest = dTest + v[7][i]*temp
        
        
        
    #xD = np.random.normal(0, math.sqrt(w[0]), numSamples).T
    #yD = np.random.normal(0, math.sqrt(w[1]), numSamples).T
    #zD = np.random.normal(0, math.sqrt(w[2]), numSamples).T
    #wD = np.random.normal(0, math.sqrt(w[3]), numSamples).T

    #xTest = v[0][0]*xD + v[0][1]*yD
    #yTest = v[1][0]*xD + v[1][1]*yD
    #zTest = v[2][0]*xD + v[2][1]*yD
    #wTest = v[3][0]*xD + v[3][1]*yD
    #aTest = v[4][0]*xD + v[4][1]*yD
    #bTest = v[5][0]*xD + v[5][1]*yD
    #cTest = v[6][0]*xD + v[6][1]*yD
    #dTest = v[7][0]*xD + v[7][1]*yD
    
    
   # xTest = v[0][0]*xD 
   # yTest = v[1][0]*xD 
   # zTest = v[2][0]*xD 
   # wTest = v[3][0]*xD 
   # aTest = v[4][0]*xD 
   # bTest = v[5][0]*xD 
   # cTest = v[6][0]*xD 
   # dTest = v[7][0]*xD 

    numNegativeNonCrossing = 0
    for i in range(numSamples):
        if ((xTest[i]<=transformedIso1) and (yTest[i]<=transformedIso2) and (zTest[i]<=transformedIso3) and (wTest[i]<=transformedIso4) and (aTest[i]<=transformedIso5) and (bTest[i]<=transformedIso6) and (cTest[i]<=transformedIso7) and (dTest[i]<=transformedIso8)):
            numNegativeNonCrossing = numNegativeNonCrossing + 1
        
    numPositiveNonCrossing = 0
    for i in range(numSamples):
        if ((xTest[i]>=transformedIso1) and (yTest[i]>=transformedIso2) and (zTest[i]>=transformedIso3) and (wTest[i]>=transformedIso4) and (aTest[i]>=transformedIso5) and (bTest[i]>=transformedIso6) and (cTest[i]>=transformedIso7) and (dTest[i]>=transformedIso8)):
            numPositiveNonCrossing = numPositiveNonCrossing + 1
    
    #q1 = (v[0][0]**2)*abs(w[0]) + (v[0][1]**2)*abs(w[1]) + (v[0][2]**2)*abs(w[2]) + (v[0][3]**2)*abs(w[3])
    #std1 = math.sqrt(q1)

    #q2 = (v[1][0]**2)*abs(w[0]) + (v[1][1]**2)*abs(w[1]) + (v[1][2]**2)*abs(w[2]) + (v[1][3]**2)*abs(w[3])
    #std2 = math.sqrt(q2)
    
    #q3 = (v[2][0]**2)*abs(w[0]) + (v[2][1]**2)*abs(w[1]) + (v[2][2]**2)*abs(w[2]) + (v[2][3]**2)*abs(w[3])
    #std3 = math.sqrt(q3)
    
    #q4 = (v[3][0]**2)*abs(w[0]) + (v[3][1]**2)*abs(w[1]) + (v[3][2]**2)*abs(w[2]) + (v[3][3]**2)*abs(w[3])
    #std4 = math.sqrt(q4)
    
    #negativeProbm1 = 0.5*(1 + math.erf((transformedIso1 - 0)/(math.sqrt(2)*std1)))
    #negativeProbm2 = 0.5*(1 + math.erf((transformedIso2 - 0)/(math.sqrt(2)*std2)))
    #negativeProbm3 = 0.5*(1 + math.erf((transformedIso3 - 0)/(math.sqrt(2)*std3)))
    #negativeProbm4 = 0.5*(1 + math.erf((transformedIso4 - 0)/(math.sqrt(2)*std4)))
    
    #positiveProbm1 = 1.0 - negativeProbm1
    #positiveProbm2 = 1.0 - negativeProbm2
    #positiveProbm3 = 1.0 - negativeProbm3
    #positiveProbm4 = 1.0 - negativeProbm4
    
    #negativeNonCrossingProb = negativeProbm1*negativeProbm2*negativeProbm3*negativeProbm4
    #positiveNonCrossingProb = positiveProbm1*positiveProbm2*positiveProbm3*positiveProbm4
    
    crossingProb = 1.0 - (numNegativeNonCrossing/numSamples) - (numPositiveNonCrossing/numSamples)
    
    #if (abs(crossingProbOriginal-crossingProb)>0.3):
    #    print('eigenvalues:', w)
    
    return crossingProb

In [31]:
def computePearsonCorrelation(ensemble):
    
    h, w, d, numMembers = ensemble.shape
    
    pearsonCoefficients = np.zeros((h-1,w-1,d-1))
    
    for i in range(h-1):
        for j in range(w-1):
            for k in range(d-1):
                
                d1 = ensemble[i,j,k,:]
                #print("d1",d1)
                d2 = ensemble[i+1,j,k,:]
                d3 = ensemble[i+1,j+1,k,:]
                d4 = ensemble[i,j+1,k,:]
                d5 = ensemble[i,j,k+1,:]
                d6 = ensemble[i+1,j,k+1,:]
                d7 = ensemble[i+1,j+1,k+1,:]
                d8 = ensemble[i,j+1,k+1,:] 
                
                # Add to avoid invalid correlation value
                #if (np.all(d1 == d1[0])):
                #    d1 = d1 + np.random.uniform(0,0.001,numMembers)
                #if (np.all(d2 == d2[0])):
                #    d2 = d2 + np.random.uniform(0,0.001,numMembers)
                #if (np.all(d3 == d3[0])):
                #    d3 = d3 + np.random.uniform(0,0.001,numMembers)
                #if (np.all(d4 == d4[0])):
                #    d4 = d4 + np.random.uniform(0,0.001,numMembers)
                #if (np.all(d5 == d5[0])):
                #    d5 = d5 + np.random.uniform(0,0.001,numMembers)
                #if (np.all(d6 == d6[0])):
                #    d6 = d6 + np.random.uniform(0,0.001,numMembers)
                #if (np.all(d7 == d7[0])):
                #    d7 = d7 + np.random.uniform(0,0.001,numMembers)
                #if (np.all(d8 == d8[0])):
                #    d8 = d8 + np.random.uniform(0,0.001,numMembers)
                
                #d1 = d1 + np.random.uniform(0,0.001,numMembers)
                #d2 = d2 + np.random.uniform(0,0.001,numMembers)
                #d3 = d3 + np.random.uniform(0,0.001,numMembers)
                #d4 = d4 + np.random.uniform(0,0.001,numMembers)
                #d5 = d5 + np.random.uniform(0,0.001,numMembers)    
                #d6 = d6 + np.random.uniform(0,0.001,numMembers)
                #d7 = d7 + np.random.uniform(0,0.001,numMembers)
                #d8 = d8 + np.random.uniform(0,0.001,numMembers)
                    
                pr1 = sc.stats.pearsonr(d1,d2)
                pr2 = sc.stats.pearsonr(d1,d3)
                pr3 = sc.stats.pearsonr(d1,d4)
                pr4 = sc.stats.pearsonr(d1,d5)
                pr5 = sc.stats.pearsonr(d1,d6)
                pr6 = sc.stats.pearsonr(d1,d7)
                pr7 = sc.stats.pearsonr(d1,d8)
                    
                pr8 = sc.stats.pearsonr(d2,d3)
                pr9 = sc.stats.pearsonr(d2,d4)
                pr10 = sc.stats.pearsonr(d2,d5)
                pr11 = sc.stats.pearsonr(d2,d6)
                pr12 = sc.stats.pearsonr(d2,d7)
                pr13 = sc.stats.pearsonr(d2,d8)
                    
                pr14 = sc.stats.pearsonr(d3,d4)
                pr15 = sc.stats.pearsonr(d3,d5)
                pr16 = sc.stats.pearsonr(d3,d6)
                pr17 = sc.stats.pearsonr(d3,d7)
                pr18 = sc.stats.pearsonr(d3,d8)
                    
                pr19 = sc.stats.pearsonr(d4,d5)
                pr20 = sc.stats.pearsonr(d4,d6)
                pr21 = sc.stats.pearsonr(d4,d7)
                pr22 = sc.stats.pearsonr(d4,d8)
                    
                pr23 = sc.stats.pearsonr(d5,d6)
                pr24 = sc.stats.pearsonr(d5,d7)
                pr25 = sc.stats.pearsonr(d5,d8)
                    
                pr26 = sc.stats.pearsonr(d6,d7)
                pr27 = sc.stats.pearsonr(d6,d8)
                    
                pr28 = sc.stats.pearsonr(d7,d8)
                
                maxPearsonCorrelation = max(abs(pr1[0]),abs(pr2[0]),abs(pr3[0]),abs(pr4[0]),abs(pr5[0]),abs(pr6[0]),abs(pr7[0]),abs(pr8[0]),abs(pr9[0]),abs(pr10[0]),abs(pr11[0]),abs(pr12[0]),abs(pr13[0]),abs(pr14[0]),abs(pr15[0]),abs(pr16[0]),abs(pr17[0]),abs(pr18[0]),abs(pr19[0]),abs(pr20[0]),abs(pr21[0]),abs(pr22[0]),abs(pr23[0]),abs(pr24[0]),abs(pr25[0]),abs(pr26[0]),abs(pr27[0]),abs(pr28[0]))
                    
                #averagePearsonCorrelation = (abs(pr1[0])+abs(pr2[0])+abs(pr3[0])+abs(pr4[0])+abs(pr5[0])+abs(pr6[0])+abs(pr7[0])+abs(pr8[0])+abs(pr9[0])+abs(pr10[0])+abs(pr11[0])+abs(pr12[0])+abs(pr13[0])+abs(pr14[0])+abs(pr15[0])+abs(pr16[0])+abs(pr17[0])+abs(pr18[0])+abs(pr19[0])+abs(pr20[0])+abs(pr21[0])+abs(pr22[0])+abs(pr23[0])+abs(pr24[0])+abs(pr25[0])+abs(pr26[0])+abs(pr27[0])+abs(pr28[0]))/28
                pearsonCoefficients[i,j,k] = maxPearsonCorrelation
                
                #maxPearsonCorrelation = np.amax(pr1[0],pr2[0],pr3[0],pr4[0],pr5[0],pr6[0],pr7[0],pr8[0],pr9[0],pr10[0],pr11[0],pr12[0],pr13[0],pr14[0],pr15[0],pr16[0],pr17[0],pr18[0],pr19[0],pr20[0],pr21[0],pr22[0],pr23[0],pr24[0],pr25[0],pr26[0],pr27[0],pr28[0])
                #pearsonCoefficients[i,j,k] = maxPearsonCorrelation
                 
                
    return pearsonCoefficients

In [33]:
def PearsonAdaptiveBasedProbabilisticMarchingCubesMultivariate(ensemble, isovalue, correlationMap):
    
    h, w, d, numMembers = ensemble.shape
    # Record number of cases per cell as well
    crossingProbabilities = np.zeros((h-1,w-1,d-1))
    
    minimumGrid = np.zeros((h,w,d))
    maximumGrid = np.zeros((h,w,d))
    
    
    for i in range(h):
        for j in range(w):
            for k in range(d):
                minimum = math.inf
                maximum = -math.inf
                for l in range(numMembers):
                    if(ensemble[i,j,k,l] < minimum):
                        minimum = ensemble[i,j,k,l]
                    if(ensemble[i,j,k,l] > maximum):
                        maximum = ensemble[i,j,k,l]    
                minimumGrid[i,j,k] = minimum
                maximumGrid[i,j,k] = maximum
            
    random.seed(10)   
    numSamples = 500
    
    pearsonCoefficients = np.zeros((h-1,w-1,d-1))
    for i in range(h-1):
        for j in range(w-1):
            for k in range(d-1):
                
                x1min = minimumGrid[i,j,k]
                x1max = maximumGrid[i,j,k]
                x2min = minimumGrid[i+1,j,k]
                x2max = maximumGrid[i+1,j,k]
                x3min = minimumGrid[i+1,j+1,k]
                x3max = maximumGrid[i+1,j+1,k]
                x4min = minimumGrid[i,j+1,k]
                x4max = maximumGrid[i,j+1,k]
                x5min = minimumGrid[i,j,k+1]
                x5max = maximumGrid[i,j,k+1]
                x6min = minimumGrid[i+1,j,k+1]
                x6max = maximumGrid[i+1,j,k+1]
                x7min = minimumGrid[i+1,j+1,k+1]
                x7max = maximumGrid[i+1,j+1,k+1]
                x8min = minimumGrid[i,j+1,k+1]
                x8max = maximumGrid[i,j+1,k+1]
            
                minCell = min(x1min, x2min, x3min, x4min, x5min, x6min, x7min, x8min)
                maxCell = max(x1max, x2max, x3max, x4max, x5min, x6min, x7min, x8min)
                
                
                # sample if isovalue falls between minCell and maxCell
                if (isovalue > minCell) and (isovalue < maxCell):
                
                    #Estimate mean and covariance matrices from samples
                    m1 = np.mean(ensemble[i,j,k,:])
                    m2 = np.mean(ensemble[i+1,j,k,:])   
                    m3 = np.mean(ensemble[i+1,j+1,k,:])
                    m4 = np.mean(ensemble[i,j+1,k,:])     
                    m5 = np.mean(ensemble[i,j,k+1,:])
                    m6 = np.mean(ensemble[i+1,j,k+1,:])   
                    m7 = np.mean(ensemble[i+1,j+1,k+1,:])
                    m8 = np.mean(ensemble[i,j+1,k+1,:]) 
                    
                    meanArray = np.array([m1,m2,m3,m4,m5,m6,m7,m8])     
                    #print(meanArray)
                
                    d1 = ensemble[i,j,k,:]
                    d2 = ensemble[i+1,j,k,:]
                    d3 = ensemble[i+1,j+1,k,:]
                    d4 = ensemble[i,j+1,k,:]
                    d5 = ensemble[i,j,k+1,:]
                    d6 = ensemble[i+1,j,k+1,:]
                    d7 = ensemble[i+1,j+1,k+1,:]
                    d8 = ensemble[i,j+1,k+1,:]
                    
                    observedData = np.array([d1,d2,d3,d4,d5,d6,d7,d8])
                    
                    #pr1 = sc.stats.pearsonr(d1,d2)
                    #pr2 = sc.stats.pearsonr(d2,d3)
                    #pr3 = sc.stats.pearsonr(d3,d4)
                    #pr4 = sc.stats.pearsonr(d4,d5)
                    #pr5 = sc.stats.pearsonr(d5,d6)
                    #pr6 = sc.stats.pearsonr(d6,d7)
                    #pr7 = sc.stats.pearsonr(d7,d8)
                    
                    #maxPearsonCorrelation = max(abs(pr1[0]),abs(pr2[0]),abs(pr3[0]),abs(pr4[0]),abs(pr5[0]),abs(pr6[0]),abs(pr7[0]))
                    
                    #pr1 = sc.stats.pearsonr(d1,d2)
                    #pr2 = sc.stats.pearsonr(d1,d3)
                    #pr3 = sc.stats.pearsonr(d1,d4)
                    #pr4 = sc.stats.pearsonr(d1,d5)
                    #pr5 = sc.stats.pearsonr(d1,d6)
                    #pr6 = sc.stats.pearsonr(d1,d7)
                    #pr7 = sc.stats.pearsonr(d1,d8)
                    
                    #pr8 = sc.stats.pearsonr(d2,d3)
                    #pr9 = sc.stats.pearsonr(d2,d4)
                    #pr10 = sc.stats.pearsonr(d2,d5)
                    #pr11 = sc.stats.pearsonr(d2,d6)
                    #pr12 = sc.stats.pearsonr(d2,d7)
                    #pr13 = sc.stats.pearsonr(d2,d8)
                    
                    #pr14 = sc.stats.pearsonr(d3,d4)
                    #pr15 = sc.stats.pearsonr(d3,d5)
                    #pr16 = sc.stats.pearsonr(d3,d6)
                    #pr17 = sc.stats.pearsonr(d3,d7)
                    #pr18 = sc.stats.pearsonr(d3,d8)
                    
                    #pr19 = sc.stats.pearsonr(d4,d5)
                    #pr20 = sc.stats.pearsonr(d4,d6)
                    #pr21 = sc.stats.pearsonr(d4,d7)
                    #pr22 = sc.stats.pearsonr(d4,d8)
                    
                    #pr23 = sc.stats.pearsonr(d5,d6)
                    #pr24 = sc.stats.pearsonr(d5,d7)
                    #pr25 = sc.stats.pearsonr(d5,d8)
                    
                    #pr26 = sc.stats.pearsonr(d6,d7)
                    #pr27 = sc.stats.pearsonr(d6,d8)
                    
                    #pr28 = sc.stats.pearsonr(d7,d8)
                    
                    #averagePearsonCorrelation = (abs(pr1[0])+abs(pr2[0])+abs(pr3[0])+abs(pr4[0])+abs(pr5[0])+abs(pr6[0])+abs(pr7[0])+abs(pr8[0])+abs(pr9[0])+abs(pr10[0])+abs(pr11[0])+abs(pr12[0])+abs(pr13[0])+abs(pr14[0])+abs(pr15[0])+abs(pr16[0])+abs(pr17[0])+abs(pr18[0])+abs(pr19[0])+abs(pr20[0])+abs(pr21[0])+abs(pr22[0])+abs(pr23[0])+abs(pr24[0])+abs(pr25[0])+abs(pr26[0])+abs(pr27[0])+abs(pr28[0]))/28
                    #pearsonCoefficients[i,j,k] = averagePearsonCorrelation
                    
                    #maxPearsonCorrelation = max(abs(pr1[0]),abs(pr2[0]),abs(pr3[0]),abs(pr4[0]),abs(pr5[0]),abs(pr6[0]),abs(pr7[0]),abs(pr8[0]),abs(pr9[0]),abs(pr10[0]),abs(pr11[0]),abs(pr12[0]),abs(pr13[0]),abs(pr14[0]),abs(pr15[0]),abs(pr16[0]),abs(pr17[0]),abs(pr18[0]),abs(pr19[0]),abs(pr20[0]),abs(pr21[0]),abs(pr22[0]),abs(pr23[0]),abs(pr24[0]),abs(pr25[0]),abs(pr26[0]),abs(pr27[0]),abs(pr28[0]))
                    #pearsonCoefficients[i,j,k] = maxPearsonCorrelation
                    
                    #pr1 = sc.stats.pearsonr(d1,d2)
                    #pr2 = sc.stats.pearsonr(d2,d3)
                    #pr3 = sc.stats.pearsonr(d3,d4)
                    #pr4 = sc.stats.pearsonr(d4,d5)
                    #pr5 = sc.stats.pearsonr(d5,d6)
                    #pr6 = sc.stats.pearsonr(d6,d7)
                    #pr7 = sc.stats.pearsonr(d7,d8)
                    #averagePearsonCorrelation = abs((pr1[0]+pr2[0]+pr3[0]+pr4[0]+pr5[0]+pr6[0]+pr7[0])/7)
                    
                    #pr1 = sc.stats.pearsonr(d1,d2)
                    #pr2 = sc.stats.pearsonr(d1,d3)
                    #pr3 = sc.stats.pearsonr(d1,d4)
                    #pr4 = sc.stats.pearsonr(d1,d5)
                    #pr5 = sc.stats.pearsonr(d1,d6)
                    #pr6 = sc.stats.pearsonr(d1,d7)
                    #pr7 = sc.stats.pearsonr(d1,d8)
                    
                    #pr8 = sc.stats.pearsonr(d2,d3)
                    #pr9 = sc.stats.pearsonr(d2,d4)
                    #pr10 = sc.stats.pearsonr(d2,d5)
                    #pr11 = sc.stats.pearsonr(d2,d6)
                    #pr12 = sc.stats.pearsonr(d2,d7)
                    #pr13 = sc.stats.pearsonr(d2,d8)
                    
                    #pr14 = sc.stats.pearsonr(d3,d4)
                    #pr15 = sc.stats.pearsonr(d3,d5)
                    #pr16 = sc.stats.pearsonr(d3,d6)
                    #pr17 = sc.stats.pearsonr(d3,d7)
                    #pr18 = sc.stats.pearsonr(d3,d8)
                    
                    #pr19 = sc.stats.pearsonr(d4,d5)
                    #pr20 = sc.stats.pearsonr(d4,d6)
                    #pr21 = sc.stats.pearsonr(d4,d7)
                    #pr22 = sc.stats.pearsonr(d4,d8)
                    
                    #pr23 = sc.stats.pearsonr(d5,d6)
                    #pr24 = sc.stats.pearsonr(d5,d7)
                    #pr25 = sc.stats.pearsonr(d5,d8)
                    
                    #pr26 = sc.stats.pearsonr(d6,d7)
                    #pr27 = sc.stats.pearsonr(d6,d8)
                    
                    #pr28 = sc.stats.pearsonr(d7,d8)
                    
                    #averagePearsonCorrelation = abs((pr1[0]+pr2[0]+pr3[0]+pr4[0]+pr5[0]+pr6[0]+pr7[0]+pr8[0]+pr9[0]+pr10[0]+pr11[0]+pr12[0]+pr13[0]+pr14[0]+pr15[0]+pr16[0]+pr17[0]+pr18[0]+pr19[0]+pr20[0]+pr21[0]+pr22[0]+pr23[0]+pr24[0]+pr25[0]+pr26[0]+pr27[0]+pr28[0])/28)
                    maxPearsonCorrelation = correlationMap[i,j,k]
                    pearsonCoefficients[i,j,k] = maxPearsonCorrelation
                    
                    if (maxPearsonCorrelation<=0.2):       
        
                        std1 = np.std(d1)
                        std2 = np.std(d2)
                        std3 = np.std(d3)
                        std4 = np.std(d4)
                        std5 = np.std(d5)
                        std6 = np.std(d6)
                        std7 = np.std(d7)
                        std8 = np.std(d8)
                        
                        # Avoid divide by 0 error
                        if (std1==0):
                            std1 = 0.00001
                        if (std2==0):
                            std2 = 0.00001
                        if (std3==0):
                            std3 = 0.00001
                        if (std4==0):
                            std4 = 0.00001
                        if (std5==0):
                            std5 = 0.00001
                        if (std6==0):
                            std6 = 0.00001
                        if (std7==0):
                            std7 = 0.00001
                        if (std8==0):
                            std8 = 0.00001
                        
                        negProb1 = 0.5*(1 + math.erf((isovalue - m1)/(math.sqrt(2)*std1)))
                        negProb2 = 0.5*(1 + math.erf((isovalue - m2)/(math.sqrt(2)*std2)))
                        negProb3 = 0.5*(1 + math.erf((isovalue - m3)/(math.sqrt(2)*std3)))
                        negProb4 = 0.5*(1 + math.erf((isovalue - m4)/(math.sqrt(2)*std4)))
                        negProb5 = 0.5*(1 + math.erf((isovalue - m5)/(math.sqrt(2)*std5)))
                        negProb6 = 0.5*(1 + math.erf((isovalue - m6)/(math.sqrt(2)*std6)))
                        negProb7 = 0.5*(1 + math.erf((isovalue - m7)/(math.sqrt(2)*std7)))
                        negProb8 = 0.5*(1 + math.erf((isovalue - m8)/(math.sqrt(2)*std8)))
        
                        posProb1 = 1 - negProb1
                        posProb2 = 1 - negProb2
                        posProb3 = 1 - negProb3
                        posProb4 = 1 - negProb4
                        posProb5 = 1 - negProb5
                        posProb6 = 1 - negProb6
                        posProb7 = 1 - negProb7
                        posProb8 = 1 - negProb8
        
                        allNegativeProb = negProb1*negProb2*negProb3*negProb4*negProb5*negProb6*negProb7*negProb8
                        allPositiveProb = posProb1*posProb2*posProb3*posProb4*posProb5*posProb6*posProb7*posProb8
        
                        crossingProb = 1 - allNegativeProb -allPositiveProb
                
                    else:
                
                        covMatrix = np.cov(observedData)
                        crossingProbEig = PearsonAdaptiveCrossingProbWithEigenDecomposition(meanArray, covMatrix, isovalue, numSamples)
                    
                    crossingProbabilities[i,j,k] = crossingProbEig
                
    return crossingProbabilities, pearsonCoefficients

In [40]:
def eigenBasedProbabilisticMarchingCubesMultivariate(ensemble, isovalue):
    
    h, w, d, numMembers = ensemble.shape
    # Record number of cases per cell as well
    crossingProbabilities = np.zeros((h-1,w-1,d-1))
    
    minimumGrid = np.zeros((h,w,d))
    maximumGrid = np.zeros((h,w,d))
    #eigenmap = np.zeros((h,w,d,8))
    
    
    for i in range(h):
        for j in range(w):
            for k in range(d):
                minimum = math.inf
                maximum = -math.inf
                for l in range(numMembers):
                    if(ensemble[i,j,k,l] < minimum):
                        minimum = ensemble[i,j,k,l]
                    if(ensemble[i,j,k,l] > maximum):
                        maximum = ensemble[i,j,k,l]    
                minimumGrid[i,j,k] = minimum
                maximumGrid[i,j,k] = maximum
            
    random.seed(10)   
    numSamples = 500
    for i in range(h-1):
        for j in range(w-1):
            for k in range(d-1):
                
                x1min = minimumGrid[i,j,k]
                x1max = maximumGrid[i,j,k]
                x2min = minimumGrid[i+1,j,k]
                x2max = maximumGrid[i+1,j,k]
                x3min = minimumGrid[i+1,j+1,k]
                x3max = maximumGrid[i+1,j+1,k]
                x4min = minimumGrid[i,j+1,k]
                x4max = maximumGrid[i,j+1,k]
                x5min = minimumGrid[i,j,k+1]
                x5max = maximumGrid[i,j,k+1]
                x6min = minimumGrid[i+1,j,k+1]
                x6max = maximumGrid[i+1,j,k+1]
                x7min = minimumGrid[i+1,j+1,k+1]
                x7max = maximumGrid[i+1,j+1,k+1]
                x8min = minimumGrid[i,j+1,k+1]
                x8max = maximumGrid[i,j+1,k+1]
            
                minCell = min(x1min, x2min, x3min, x4min, x5min, x6min, x7min, x8min)
                maxCell = max(x1max, x2max, x3max, x4max, x5min, x6min, x7min, x8min)
                
                
                # sample if isovalue falls between minCell and maxCell
                if (isovalue > minCell) and (isovalue < maxCell):
                
                    #Estimate mean and covariance matrices from samples
                    m1 = np.mean(ensemble[i,j,k,:])
                    m2 = np.mean(ensemble[i+1,j,k,:])   
                    m3 = np.mean(ensemble[i+1,j+1,k,:])
                    m4 = np.mean(ensemble[i,j+1,k,:])     
                    m5 = np.mean(ensemble[i,j,k+1,:])
                    m6 = np.mean(ensemble[i+1,j,k+1,:])   
                    m7 = np.mean(ensemble[i+1,j+1,k+1,:])
                    m8 = np.mean(ensemble[i,j+1,k+1,:]) 
                    
                    meanArray = np.array([m1,m2,m3,m4,m5,m6,m7,m8])     
                    #print(meanArray)
                
                    d1 = ensemble[i,j,k,:]
                    d2 = ensemble[i+1,j,k,:]
                    d3 = ensemble[i+1,j+1,k,:]
                    d4 = ensemble[i,j+1,k,:]
                    d5 = ensemble[i,j,k+1,:]
                    d6 = ensemble[i+1,j,k+1,:]
                    d7 = ensemble[i+1,j+1,k+1,:]
                    d8 = ensemble[i,j+1,k+1,:]
                    
                    observedData = np.array([d1,d2,d3,d4,d5,d6,d7,d8])   
                
                    covMatrix = np.cov(observedData)
                    
                    # Draw samples from multivariate normal
                    #samples = np.random.multivariate_normal(meanArray, covMatrix, numSamples)
                                
                    #x1Samples = samples[:,0]
                    #x2Samples = samples[:,1]
                    #x3Samples = samples[:,2]
                    #x4Samples = samples[:,3]
                    #x5Samples = samples[:,4]
                    #x6Samples = samples[:,5]
                    #x7Samples = samples[:,6]
                    #x8Samples = samples[:,7]
                
                    #x1Samples = x1Samples.reshape(1, numSamples)
                    #x2Samples = x2Samples.reshape(1, numSamples)
                    #x3Samples = x3Samples.reshape(1, numSamples)
                    #x4Samples = x4Samples.reshape(1, numSamples)
                    #x5Samples = x5Samples.reshape(1, numSamples)
                    #x6Samples = x6Samples.reshape(1, numSamples)
                    #x7Samples = x7Samples.reshape(1, numSamples)
                    #x8Samples = x8Samples.reshape(1, numSamples)
                
                    #crossings = 0
                
                    #for l in range(numSamples):
                    #    minVal = min(x1Samples[0,l],x2Samples[0,l],x3Samples[0,l],x4Samples[0,l],x5Samples[0,l],x6Samples[0,l],x7Samples[0,l],x8Samples[0,l])
                    #    maxVal = max(x1Samples[0,l],x2Samples[0,l],x3Samples[0,l],x4Samples[0,l],x5Samples[0,l],x6Samples[0,l],x7Samples[0,l],x8Samples[0,l])
                    #    if (isovalue > minVal) and (isovalue < maxVal):
                    #        crossings = crossings + 1;
                        
                    #crossingProb = crossings/numSamples
                    
                    #crossingProbEig = getCrossingProbWithEigenDecomposition(meanArray, covMatrix, isovalue, numSamples)
                
                    crossingProbEig = adaptiveCrossingProbWithEigenDecomposition(meanArray, covMatrix, isovalue, numSamples)
                    crossingProbabilities[i,j,k] = crossingProbEig
                    #eigenmap[i,j,k,:] = eigvals
                
    return crossingProbabilities

In [13]:
def probabilisticMarchingCubesMultivariate(ensemble, isovalue):
    
    h, w, d, numMembers = ensemble.shape
    # Record number of cases per cell as well
    crossingProbabilities = np.zeros((h-1,w-1,d-1))
    
    minimumGrid = np.zeros((h,w,d))
    maximumGrid = np.zeros((h,w,d))
    for i in range(h):
        for j in range(w):
            for k in range(d):
                minimum = math.inf
                maximum = -math.inf
                for l in range(numMembers):
                    if(ensemble[i,j,k,l] < minimum):
                        minimum = ensemble[i,j,k,l]
                    if(ensemble[i,j,k,l] > maximum):
                        maximum = ensemble[i,j,k,l]    
                minimumGrid[i,j,k] = minimum
                maximumGrid[i,j,k] = maximum
            
    random.seed(10)   
    numSamples = 500
    for i in range(h-1):
        for j in range(w-1):
            for k in range(d-1):
                
                x1min = minimumGrid[i,j,k]
                x1max = maximumGrid[i,j,k]
                x2min = minimumGrid[i+1,j,k]
                x2max = maximumGrid[i+1,j,k]
                x3min = minimumGrid[i+1,j+1,k]
                x3max = maximumGrid[i+1,j+1,k]
                x4min = minimumGrid[i,j+1,k]
                x4max = maximumGrid[i,j+1,k]
                x5min = minimumGrid[i,j,k+1]
                x5max = maximumGrid[i,j,k+1]
                x6min = minimumGrid[i+1,j,k+1]
                x6max = maximumGrid[i+1,j,k+1]
                x7min = minimumGrid[i+1,j+1,k+1]
                x7max = maximumGrid[i+1,j+1,k+1]
                x8min = minimumGrid[i,j+1,k+1]
                x8max = maximumGrid[i,j+1,k+1]
            
                minCell = min(x1min, x2min, x3min, x4min, x5min, x6min, x7min, x8min)
                maxCell = max(x1max, x2max, x3max, x4max, x5min, x6min, x7min, x8min)
                
                
                # sample if isovalue falls between minCell and maxCell
                if (isovalue > minCell) and (isovalue < maxCell):
                
                    #Estimate mean and covariance matrices from samples
                    m1 = np.mean(ensemble[i,j,k,:])
                    m2 = np.mean(ensemble[i+1,j,k,:])   
                    m3 = np.mean(ensemble[i+1,j+1,k,:])
                    m4 = np.mean(ensemble[i,j+1,k,:])     
                    m5 = np.mean(ensemble[i,j,k+1,:])
                    m6 = np.mean(ensemble[i+1,j,k+1,:])   
                    m7 = np.mean(ensemble[i+1,j+1,k+1,:])
                    m8 = np.mean(ensemble[i,j+1,k+1,:]) 
                    
                    meanArray = np.array([m1,m2,m3,m4,m5,m6,m7,m8])     
                    #print(meanArray)
                
                    d1 = ensemble[i,j,k,:]
                    d2 = ensemble[i+1,j,k,:]
                    d3 = ensemble[i+1,j+1,k,:]
                    d4 = ensemble[i,j+1,k,:]
                    d5 = ensemble[i,j,k+1,:]
                    d6 = ensemble[i+1,j,k+1,:]
                    d7 = ensemble[i+1,j+1,k+1,:]
                    d8 = ensemble[i,j+1,k+1,:]
                    
                    observedData = np.array([d1,d2,d3,d4,d5,d6,d7,d8])   
                
                    covMatrix = np.cov(observedData)
                    
                    # Draw samples from multivariate normal
                    samples = np.random.multivariate_normal(meanArray, covMatrix, numSamples)
                                
                    x1Samples = samples[:,0]
                    x2Samples = samples[:,1]
                    x3Samples = samples[:,2]
                    x4Samples = samples[:,3]
                    x5Samples = samples[:,4]
                    x6Samples = samples[:,5]
                    x7Samples = samples[:,6]
                    x8Samples = samples[:,7]
                
                    x1Samples = x1Samples.reshape(1, numSamples)
                    x2Samples = x2Samples.reshape(1, numSamples)
                    x3Samples = x3Samples.reshape(1, numSamples)
                    x4Samples = x4Samples.reshape(1, numSamples)
                    x5Samples = x5Samples.reshape(1, numSamples)
                    x6Samples = x6Samples.reshape(1, numSamples)
                    x7Samples = x7Samples.reshape(1, numSamples)
                    x8Samples = x8Samples.reshape(1, numSamples)
                
                    crossings = 0
                
                    for l in range(numSamples):
                        minVal = min(x1Samples[0,l],x2Samples[0,l],x3Samples[0,l],x4Samples[0,l],x5Samples[0,l],x6Samples[0,l],x7Samples[0,l],x8Samples[0,l])
                        maxVal = max(x1Samples[0,l],x2Samples[0,l],x3Samples[0,l],x4Samples[0,l],x5Samples[0,l],x6Samples[0,l],x7Samples[0,l],x8Samples[0,l])
                        if (isovalue > minVal) and (isovalue < maxVal):
                            crossings = crossings + 1;
                        
                    crossingProb = crossings/numSamples
                
                    crossingProbabilities[i,j,k] = crossingProb
                
    return crossingProbabilities

In [20]:
# Visualize crossing probabilities : Probabilistic marching squares
isovalue = 900
t1_start = process_time() 
crossingProbabilitiesMultivariate = probabilisticMarchingCubesMultivariate(ensemble, isovalue)
t1_stop = process_time()
print('multivariate time:', t1_stop-t1_start)

multivariate time: 383.383049


In [42]:
# Visualize crossing probabilities : Probabilistic marching squares
isovalue = 900
t1_start = process_time() 
eigenCrossingProbabilitiesMultivariate = eigenBasedProbabilisticMarchingCubesMultivariate(ensemble, isovalue)
t1_stop = process_time()
print('eigen-based multivariate time:', t1_stop-t1_start)

/var/folders/1c/3fmc6kxd5kzcmjws0d7kmhw85hdtjh/T/ipykernel_62857/1365307529.py:59: ComplexWarning: Casting complex values to real discards the imaginary part
  temp = np.random.normal(0, math.sqrt(w[i]), numSamples).T


eigen-based multivariate time: 240.4232839999995


In [53]:
# Precompute Pearson's correlation map
pearsonCorr= computePearsonCorrelation(ensemble)


/usr/local/anaconda3/envs/marchingCubesTopoUncertainty/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


In [55]:
print(pearsonCorr.shape)

(123, 207, 207)


In [34]:
# Visualize crossing probabilities : Probabilistic marching squares (Pearson)
isovalue = 900
t1_start = process_time()
crossingProbabilitiesPearson, pearsoncoeff = PearsonAdaptiveBasedProbabilisticMarchingCubesMultivariate(ensemble, isovalue, pearsonCorr)
t1_stop = process_time()
print('Pearson time:', t1_stop-t1_start)

/var/folders/1c/3fmc6kxd5kzcmjws0d7kmhw85hdtjh/T/ipykernel_62857/1878007764.py:60: ComplexWarning: Casting complex values to real discards the imaginary part
  temp = np.random.normal(0, math.sqrt(w[i]), numSamples).T


Pearson time: 231.28344900000002


In [56]:
imageToVTK("./maxPearsonCorrelation", pointData = {"maxPearsonCorrelation" : pearsonCorr})

'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/maxPearsonCorrelation.vti'

In [74]:
imageToVTK("./multivariateGaussianCrossingProb", pointData = {"multivariateGaussianCrossingProb" : crossingProbabilitiesMultivariate})

'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/multivariateGaussianCrossingProb.vti'

In [43]:
imageToVTK("./eigenMultivariateGaussianCrossingProb", pointData = {"eigenMultivariateGaussianCrossingProb" : eigenCrossingProbabilitiesMultivariate})

'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenMultivariateGaussianCrossingProb.vti'

In [37]:
imageToVTK("./pearsonMultivariateGaussianCrossingProb", pointData = {"pearsonMultivariateGaussianCrossingProb" : crossingProbabilitiesPearson})

'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/pearsonMultivariateGaussianCrossingProb.vti'

In [76]:
print(eigenmap.shape)
eigenVal0 = np.squeeze(eigenmap[:,:,:,0])
eigenVal0 = eigenVal0.astype(float)
print(eigenVal0.shape)
imageToVTK("./eigenvalue0", pointData = {"eigenvalue0" : eigenVal0})

(123, 208, 208, 8)
(123, 208, 208)


'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenvalue0.vti'

In [77]:
print(eigenmap.shape)
eigenVal1 = np.squeeze(eigenmap[:,:,:,1])
eigenVal1 = eigenVal1.astype(float)
print(eigenVal1.shape)
imageToVTK("./eigenvalue1", pointData = {"eigenvalue1" : eigenVal1})

(123, 208, 208, 8)
(123, 208, 208)


'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenvalue1.vti'

In [78]:
print(eigenmap.shape)
eigenVal2 = np.squeeze(eigenmap[:,:,:,2])
eigenVal2 = eigenVal2.astype(float)
print(eigenVal2.shape)
imageToVTK("./eigenvalue2", pointData = {"eigenvalue2" : eigenVal2})

(123, 208, 208, 8)
(123, 208, 208)


'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenvalue2.vti'

In [79]:
print(eigenmap.shape)
eigenVal3 = np.squeeze(eigenmap[:,:,:,3])
eigenVal3 = eigenVal3.astype(float)
print(eigenVal3.shape)
imageToVTK("./eigenvalue3", pointData = {"eigenvalue3" : eigenVal3})

(123, 208, 208, 8)
(123, 208, 208)


'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenvalue3.vti'

In [80]:
print(eigenmap.shape)
eigenVal4 = np.squeeze(eigenmap[:,:,:,4])
eigenVal4 = eigenVal4.astype(float)
print(eigenVal4.shape)
imageToVTK("./eigenvalue4", pointData = {"eigenvalue4" : eigenVal4})

(123, 208, 208, 8)
(123, 208, 208)


'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenvalue4.vti'

In [81]:
print(eigenmap.shape)
eigenVal5 = np.squeeze(eigenmap[:,:,:,5])
eigenVal5 = eigenVal5.astype(float)
print(eigenVal5.shape)
imageToVTK("./eigenvalue5", pointData = {"eigenvalue5" : eigenVal5})

(123, 208, 208, 8)
(123, 208, 208)


'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenvalue5.vti'

In [82]:
print(eigenmap.shape)
eigenVal6 = np.squeeze(eigenmap[:,:,:,6])
eigenVal6 = eigenVal6.astype(float)
print(eigenVal6.shape)
imageToVTK("./eigenvalue6", pointData = {"eigenvalue6" : eigenVal6})

(123, 208, 208, 8)
(123, 208, 208)


'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenvalue6.vti'

In [83]:
print(eigenmap.shape)
eigenVal7 = np.squeeze(eigenmap[:,:,:,7])
eigenVal7 = eigenVal7.astype(float)
print(eigenVal7.shape)
imageToVTK("./eigenvalue7", pointData = {"eigenvalue7" : eigenVal7})

(123, 208, 208, 8)
(123, 208, 208)


'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/eigenvalue7.vti'

In [46]:
#error = abs(crossingProbabilitiesPearson-crossingProbabilitiesMultivariate)
error = abs(eigenCrossingProbabilitiesMultivariate-crossingProbabilitiesMultivariate)
h,w,d = eigenCrossingProbabilitiesMultivariate.shape
for i in range(h):
    for j in range(w):
        for k in range(d):
            if (error[i,j,k] > 0.1):
                print('error:',error[i,j,k])

error: 0.10200000000000009
error: 0.10599999999999998
error: 0.10000000000000003
error: 0.10999999999999999
error: 0.10799999999999998
error: 0.11199999999999999
error: 0.11199999999999993
error: 0.10399999999999998
error: 0.10200000000000004
error: 0.11199999999999999
error: 0.1060000000000001
error: 0.10199999999999998
error: 0.10199999999999998
error: 0.10599999999999998
error: 0.10599999999999998
error: 0.10400000000000004
error: 0.11599999999999999
error: 0.10200000000000004
error: 0.10799999999999998
error: 0.12000000000000005
error: 0.10200000000000004
error: 0.10199999999999998
error: 0.1100000000000001
error: 0.10399999999999998
error: 0.10000000000000003
error: 0.12
error: 0.10800000000000004
error: 0.10999999999999999
error: 0.128
error: 0.11599999999999999
error: 0.10599999999999998
error: 0.10600000000000004
error: 0.11200000000000004
error: 0.10199999999999998
error: 0.10399999999999998
error: 0.12
error: 0.10000000000000003
error: 0.118
error: 0.11199999999999999
error: 

In [47]:
imageToVTK("./diffEigenVsOriginalMultivariateGaussianCrossingProb", pointData = {"diffEigenVsOriginalMultivariateGaussianCrossingProb" : abs(eigenCrossingProbabilitiesMultivariate-crossingProbabilitiesMultivariate)})

'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/diffEigenVsOriginalMultivariateGaussianCrossingProb.vti'

In [38]:
imageToVTK("./diffPearsonVsOriginalMultivariateGaussianCrossingProb", pointData = {"diffPearsonVsOriginalMultivariateGaussianCrossingProb" : abs(crossingProbabilitiesPearson-crossingProbabilitiesMultivariate)})

'/Users/tm8/tusharathawale/projects/research/multivariateGaussianClosedFormCrossingProb/3D/diffPearsonVsOriginalMultivariateGaussianCrossingProb.vti'

In [6]:
isovalue = 900
[positiveProbabilitiesGrid, negativeProbabilitiesGrid,  maximumGrid, minimumGrid, mostProbableGrid] = computePositiveNegativeProbabilities(ensemble, isovalue, 100, 'Gaussian')
#plt.figure()
#plt.imshow(positiveProbabilitiesGrid)
#plt.contour(positiveProbabilitiesGrid, levels=[0.5])
#MStopologyCasesProbabilities,entropy,numCases = computeMSTopologyCasesProbabilities(positiveProbabilitiesGrid, negativeProbabilitiesGrid)
MStopologyCasesProbabilities,entropy,numCases = computeMSTopologyCasesProbabilities(positiveProbabilitiesGrid, negativeProbabilitiesGrid)

In [ ]:
MStopologyCasesProbabilities,entropy,numCases = computeMSTopologyCasesProbabilities(positiveProbabilitiesGrid, negativeProbabilitiesGrid)



In [8]:
imageToVTK("./mostProbableGrid", pointData = {"mostProbableGrid" : mostProbableGrid})
imageToVTK("./entropy", pointData = {"entropy" : entropy})
imageToVTK("./numCases", pointData = {"numCases" : numCases})

'/Users/tusharathawale/postdoc/marchingCubesTopologyUncertaintyVis/vis21/code/3D/numCases.vti'

In [12]:
crossingProbabilities = probabilisticMarchingSquares(positiveProbabilitiesGrid, negativeProbabilitiesGrid)
imageToVTK("./crossingProbabilities", pointData = {"crossingProbabilities" : crossingProbabilities})

'/Users/tusharathawale/postdoc/marchingCubesTopologyUncertaintyVis/vis21/code/3D/crossingProbabilities.vti'

In [33]:
# Test single instance for isovalue 900
instance = np.zeros((123,208,208))
for i in range(123):
    for j in range(208):
        for k in range(208):
            instance[i,j,k] = ensemble[i,j,k,0]

print(instance.shape)
print(instance.flags)
print(ground_truth.flags)
print(ensemble.flags)
imageToVTK("./instance", pointData = {"instance" : instance})


(123, 208, 208)
  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : False
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False
  UPDATEIFCOPY : False



'/Users/tusharathawale/postdoc/marchingCubesTopologyUncertaintyVis/vis21/code/3D/instance.vti'

In [11]:
imageToVTK("./ground_truth", pointData = {"ground_truth" : ground_truth})

'/Users/tusharathawale/postdoc/marchingCubesTopologyUncertaintyVis/vis21/code/3D/ground_truth.vti'

In [56]:
covMatrix = 1*np.identity(8)
covMatrix[0,1] = 200
covMatrix[1,0] = 200
covMatrix[0,2] = 200
covMatrix[2,0] = 200
covMatrix[0,3] = 200
covMatrix[3,0] = 200
covMatrix[0,4] = 200
covMatrix[4,0] = 200
covMatrix[0,5] = 200
covMatrix[5,0] = 200
covMatrix[0,6] = 200
covMatrix[6,0] = 200
covMatrix[0,7] = 200
covMatrix[7,0] = 200
#covMatrix[1,3] = 200
#covMatrix[3,1] = 200

w, v = LA.eig(covMatrix)

In [57]:
print(w)

[ 530.15026221 -528.15026221    1.            1.            1.
    1.            1.            1.        ]


In [293]:
#Data

#d1 = [1,5,1,1,1,1,1,1]
#d2 = [2,10,2,2,2,2,2,2]
#d3 = [0.5,2.5,0.5,0.5,0.5,0.5,0.5,0.5]
#d4 = np.random.rand(1,8)[0]

#d1 = 300*np.random.rand(1,8)[0]
#d2 = 300*np.random.rand(1,8)[0]
#d3 = 300*np.random.rand(1,8)[0]
#d4 = 300*np.random.rand(1,8)[0]


d1 = 100*np.random.rand(1,8)[0]
d2 = 2*d1
d3 = 0.5*d1
d4 = 0.2*d1

#d1 = [1,5,2,4,3,6,7,0]
#d2 = [3,1,4,2,7,0,6,5]
#d3 = [2,5,6,3,0,1,3,4]
#d4 = [7,0,1,2,5,4,6,3]

#d1 = [1,5,2,4,3,6,7,0]
#d2 = [2, 10, 4,8,6,12,14,0]
#d3 =[3,15,6,12,9,18,21,0]
#d4 = [7,0,1,2,5,4,6,3]

#mu, sigma = 0, 1 
#d1 = np.random.normal(mu, sigma, (1,8))[0]
#d2 = 2*d1
#d3 = np.random.normal(mu, sigma, (1,8))[0]
#d4 = np.random.normal(mu, sigma, (1,8))[0]

#mu, sigma = 0, 1 
#d1 = np.random.normal(mu, sigma, (1,8))[0]
#d2 = np.random.normal(mu, sigma, (1,8))[0]
#d3 = np.random.normal(mu, sigma, (1,8))[0]
#d4 = np.random.normal(mu, sigma, (1,8))[0]

observedData = np.array([d1,d2,d3,d4]) 
covMatrix = np.cov(observedData)

In [294]:
w, v = LA.eig(covMatrix)
print(w)
#print(v)

[0.00000000e+00 6.84603668e+03 3.15544362e-30 2.12825770e-14]


In [131]:
print(np.random.rand(1,4)[0])

[0.09610035 0.68705435 0.32345864 0.40630435]


In [295]:
# Draw samples from multivariate normal
#isovalue = 2
isovalue = 50
numSamples = 10000
meanArray = [np.mean(d1), np.mean(d2), np.mean(d3), np.mean(d4)]
samples = np.random.multivariate_normal(meanArray, covMatrix, numSamples)
                                
x1Samples = samples[:,0]
x2Samples = samples[:,1]
x3Samples = samples[:,2]
x4Samples = samples[:,3]

x1Samples = x1Samples.reshape(1, numSamples)
x2Samples = x2Samples.reshape(1, numSamples)
x3Samples = x3Samples.reshape(1, numSamples)
x4Samples = x4Samples.reshape(1, numSamples)

crossings = 0
for l in range(numSamples):
    minVal = min(x1Samples[0,l],x2Samples[0,l],x3Samples[0,l],x4Samples[0,l])
    maxVal = max(x1Samples[0,l],x2Samples[0,l],x3Samples[0,l],x4Samples[0,l])
    if (isovalue > minVal) and (isovalue < maxVal):
        crossings = crossings + 1;
        
crossingProb = crossings/numSamples
print(crossingProb)

0.8156


In [296]:
# Independent normal
tempMean = [np.mean(d1), np.mean(d2), np.mean(d3), np.mean(d4)]
tempStd = [np.std(d1), np.std(d2), np.std(d3), np.std(d4)]

#print(tempMean[0])

negativeProb1 = 0.5*(1 + math.erf((isovalue - tempMean[0])/(math.sqrt(2)*tempStd[0])))
negativeProb2 = 0.5*(1 + math.erf((isovalue - tempMean[1])/(math.sqrt(2)*tempStd[1])))
negativeProb3 = 0.5*(1 + math.erf((isovalue - tempMean[2])/(math.sqrt(2)*tempStd[2])))
negativeProb4 = 0.5*(1 + math.erf((isovalue - tempMean[3])/(math.sqrt(2)*tempStd[3])))

allNegativeProb = negativeProb1*negativeProb2*negativeProb3*negativeProb4

positiveProb1 = 1.0 - negativeProb1
positiveProb2 = 1.0 - negativeProb2
positiveProb3 = 1.0 - negativeProb3
positiveProb4 = 1.0 - negativeProb4

allPositiveProb = positiveProb1*positiveProb2*positiveProb3*positiveProb4

crossingProbIndependent = 1.0 - allNegativeProb - allPositiveProb

print(crossingProbIndependent)

0.9402186372533726
